In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.examples.tutorials.mnist import input_data

/home/tsuman/Program_Files/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mnist = input_data.read_data_sets("../../Tensorflow-Bootcamp-master/03-Convolutional-Neural-Networks/MNIST_data/",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../Tensorflow-Bootcamp-master/03-Convolutional-Neural-Networks/MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../Tensorflow-Bootcamp-master/03-Convolutional-Neural-Networks/MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../../Tensorflow-Bootcamp-master/03-Convolutional-Neural-Networks/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../../Tensorflow-Bootcamp-master/03-Convolutional-Neural-Networks/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
tf.reset_default_graph() 

In [4]:
# DEFINE parameters
num_label = 10
label_hid1 = 5
label_hid2 = 3
label_hid3 = 2
label_hid4 = 1

num_image = 784 # 28*28
image_hid1 = 392
image_hid2 = 196
image_hid3 = 98
image_hid4 = 49
image_hid5 = 25
image_hid6 = 12
image_hid7 = 6
image_hid8 = 3
image_hid9 = 2
image_hid10 = 1

In [5]:
tie_weight = tf.transpose
act_func = lambda X:X
initializer = tf.variance_scaling_initializer()

In [6]:
## PLACEHOLDER DEFINATION
X = tf.placeholder(tf.float32, shape=[None, num_image])
Y = tf.placeholder(tf.float32, shape=[None, num_label])

## Loading Encoded Model

In [7]:
## IMAGE COMPRESSION
wi1 = tf.Variable(np.load('../save/weights/w1.npy'))
wi2 = tf.Variable(np.load('../save/weights/w2.npy'))
wi3 = tf.Variable(np.load('../save/weights/w3.npy'))
wi4 = tf.Variable(np.load('../save/weights/w4.npy'))
wi5 = tf.Variable(np.load('../save/weights/w5.npy'))
wi6 = tf.Variable(np.load('../save/weights/w6.npy'))
wi7 = tf.Variable(np.load('../save/weights/w7.npy'))
wi8 = tf.Variable(np.load('../save/weights/w8.npy'))
wi9 = tf.Variable(np.load('../save/weights/w9.npy'))
wi10 = tf.Variable(np.load('../save/weights/w10.npy'))

In [8]:
img_layer1 = act_func(tf.matmul(X, wi1))
img_layer2 = act_func(tf.matmul(img_layer1, wi2))
img_layer3 = act_func(tf.matmul(img_layer2, wi3))
img_layer4 = act_func(tf.matmul(img_layer3, wi4))
img_layer5 = act_func(tf.matmul(img_layer4, wi5))
img_layer6 = act_func(tf.matmul(img_layer5, wi6))
img_layer7 = act_func(tf.matmul(img_layer6, wi7))
img_layer8 = act_func(tf.matmul(img_layer7, wi8))
img_layer9 = act_func(tf.matmul(img_layer8, wi9))
img_layer10 = act_func(tf.matmul(img_layer9, wi10))

In [9]:
## LABEL DECOMPRESSION
wl1 = tf.Variable(np.load('../save/weights/lw1.npy'))
wl2 = tf.Variable(np.load('../save/weights/lw2.npy'))
wl3 = tf.Variable(np.load('../save/weights/lw3.npy'))
wl4 = tf.Variable(np.load('../save/weights/lw4.npy'))

In [10]:
lbl_layer1 = act_func(tf.matmul(Y, wl1))
lbl_layer2 = act_func(tf.matmul(lbl_layer1, wl2))
lbl_layer3 = act_func(tf.matmul(lbl_layer2, wl3))
lbl_layer4 = act_func(tf.matmul(lbl_layer3, wl4))

## Setting up learning Model

In [21]:
# Learning Parameters AKA neurogenesis
# w11 = tf.Variable(initializer([image_hid10, label_hid4]), dtype=tf.float32)

w11 = tf.Variable(np.load('../save/weights/ngw1-1.npy'))
w21 = tf.Variable(initializer([image_hid9, label_hid4]), dtype=tf.float32)


In [22]:
inpt = img_layer9
output = tf.matmul(inpt,w21)
yval = lbl_layer4

In [24]:
w21

<tf.Variable 'Variable_16:0' shape=(2, 1) dtype=float32_ref>

## Create Encode-Decode Model

In [25]:
out_layer1 = tf.matmul(output,tf.transpose(wl4))
out_layer2 = tf.matmul(out_layer1,tf.transpose(wl3))
out_layer3 = tf.matmul(out_layer2,tf.transpose(wl2))
out_layer4 = tf.matmul(out_layer3,tf.transpose(wl1))

In [26]:
# LOSS FUNCTIONS
loss = tf.reduce_mean(tf.square(output - yval))
# loss = tf.reduce_mean(tf.abs(output_layer - X))
# cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y,logits=yout))

In [33]:
# TRAINING
learning_rate = 0.0000000001
optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss, var_list=[w21])

In [34]:
# PREPARATION
init = tf.global_variables_initializer()
saver = tf.train.Saver() 

In [38]:
num_epochs = 100
batch_size = 50

try:
    with tf.Session() as sess:
        sess.run(init)

        # Epoch == Entire Training Set
        for epoch in range(num_epochs):
            num_batches = mnist.train.num_examples // batch_size

            # 150 batch size
            for iteration in range(num_batches):
                X_batch, y_batch = mnist.train.next_batch(batch_size)
                sess.run(train, feed_dict={X: X_batch, Y: y_batch})

            training_loss = loss.eval(feed_dict={X: X_batch, Y: y_batch})   
            print("Epoch {} Complete. Training Loss: {}".format(epoch,training_loss))
            saver.save(sess, "../save/neuro_2-1.ckpt")      
except KeyboardInterrupt:
    print('Keyboard Interrupted')
finally:
    print('Finished Training the Model')

Epoch 0 Complete. Training Loss: 24.54230308532715
Epoch 1 Complete. Training Loss: 25.942344665527344
Epoch 2 Complete. Training Loss: 26.583271026611328
Epoch 3 Complete. Training Loss: 31.02654266357422
Epoch 4 Complete. Training Loss: 27.170255661010742
Epoch 5 Complete. Training Loss: 22.870634078979492
Epoch 6 Complete. Training Loss: 26.177858352661133
Epoch 7 Complete. Training Loss: 31.627412796020508
Epoch 8 Complete. Training Loss: 21.45294761657715
Epoch 9 Complete. Training Loss: 30.2186336517334
Epoch 10 Complete. Training Loss: 27.086145401000977
Epoch 11 Complete. Training Loss: 33.61922836303711
Epoch 12 Complete. Training Loss: 28.28729820251465
Epoch 13 Complete. Training Loss: 24.096851348876953
Epoch 14 Complete. Training Loss: 31.706043243408203
Epoch 15 Complete. Training Loss: 25.455917358398438
Epoch 16 Complete. Training Loss: 28.037437438964844
Epoch 17 Complete. Training Loss: 29.779922485351562
Epoch 18 Complete. Training Loss: 26.771339416503906
Epoch 19 C

In [39]:
num_test_labels = 10
start_point = 15
with tf.Session() as sess:
    
    saver.restore(sess,"../save/neuro_2-1.ckpt")
    
    weight= w21.eval()
    
    matches = tf.equal(tf.argmax(out_layer4, 1), tf.argmax(Y,1))
    acc = tf.reduce_mean(tf.cast(matches, tf.float32))
    
    print('TRAIN ACCURACY: ')
    print (sess.run(acc, feed_dict={X:mnist.train.images, Y:mnist.train.labels}))

    print('TEST ACCURACY: ')
    print (sess.run(acc, feed_dict={X:mnist.test.images, Y:mnist.test.labels}))
    print('\n')

INFO:tensorflow:Restoring parameters from ../save/neuro_2-1.ckpt
TRAIN ACCURACY: 
0.10250909
TEST ACCURACY: 
0.101




In [20]:
weight

array([[0.23870963]], dtype=float32)

In [26]:
print(weight.shape)
np.save('../save/weights/ngw1-1.npy',weight)

(1, 1)
